## Day_4

In [37]:
!pip install transformers

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
text = """

English and CAPITALIZATION

🎵鸟
show_tokens False None elif == >= else: two tabs:" " Three tabs: "   "

12.0*50=600

"""

g:\Step_internship\STEP_INTERNSHIP\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [38]:
colors_list = [
    '102;194;165', '252;141;98', '141;160;203',
    '231;138;195', '166;216;84', '255;217;47'
]

def show_tokens(sentence, tokenizer_name):
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
    token_ids = tokenizer(sentence).input_ids
    for idx, t in enumerate(token_ids):
        print(
            f'\x1b[0;30;48;2;{colors_list[idx % len(colors_list)]}m' +
            tokenizer.decode(t) +
            '\x1b[0m',
            end=' '
        )

In [39]:
show_tokens(text, 'bert-base-uncased')

#WordPiece

[CLS] english and capital ##ization [UNK] [UNK] show _ token ##s false none eli ##f = = > = else : two tab ##s : " " three tab ##s : " " 12 . 0 * 50 = 600 [SEP] 

In [40]:
show_tokens(text, 'bert-base-cased')
#WordPiece

[CLS] English and CA ##PI ##TA ##L ##I ##Z ##AT ##ION [UNK] [UNK] show _ token ##s F ##als ##e None el ##if = = > = else : two ta ##bs : " " Three ta ##bs : " " 12 . 0 * 50 = 600 [SEP] 

In [41]:
show_tokens(text, 'gpt2')
#BPE


 
 English  and  CAP ITAL IZ ATION 
 
 � � � � � � 
 show _ t ok ens  False  None  el if  ==  >=  else :  two  tabs :"  "  Three  tabs :  "      " 
 
 12 . 0 * 50 = 600 

 

In [42]:
show_tokens(text, 'google/flan-t5-xxl')
#SentencePiece

English and CA PI TAL IZ ATION  <unk> show _ to ken s Fal s e None  e l if = = > = else : two tab s : " " Three tab s : " " 12. 0 * 50 = 600  </s> 

In [43]:
show_tokens(text, 'deepseek-ai/deepseek-llm-7b-chat')
#SentencePiece

<｜begin▁of▁sentence｜> 
 
 English  and  CAPITAL IZATION 
 
 � � � 鸟 
 show _ tokens  False  None  elif  ==  >=  else :  two  tabs :"  "  Three  tabs :  "     " 
 
 1 2 . 0 * 5 0 = 6 0 0 
 
 

## finding similarity with bert embedding model


In [44]:
!pip install transformers sentence-transformers

In [45]:
# prompt: generate the code that should use bert for embedding for the sentiment analysis and by using cosine similarity, i need know the  similarity for that
from transformers import BertTokenizer, BertModel
from sentence_transformers import util
import torch

# Load pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'  # You can choose other BERT models
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

def get_bert_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)  # Average pooling of token embeddings
    return embeddings

def calculate_similarity(text1, text2):
    embedding1 = get_bert_embedding(text1)
    embedding2 = get_bert_embedding(text2)

    # Calculate cosine similarity manually
    cosine_similarity = torch.nn.functional.cosine_similarity(embedding1, embedding2)
    return cosine_similarity.item()

# Example usage
text1 = "This is a positive sentence."
text2 = "This is another positive sentence."
text3 = "This is a negative sentence."

similarity_1_2 = calculate_similarity(text1, text2)
similarity_1_3 = calculate_similarity(text1, text3)

print(f"Similarity between '{text1}' and '{text2}': {similarity_1_2}")
print(f"Similarity between '{text1}' and '{text3}': {similarity_1_3}")


ImportError: cannot import name 'GenerationMixin' from 'transformers.generation' (g:\Step_internship\STEP_INTERNSHIP\.venv\Lib\site-packages\transformers\generation\__init__.py)

## Finding similarity between words after embedding with modernbert model

In [6]:
#!pip install transformers sentence-transformers

from transformers import PreTrainedTokenizerFast, BertModel
from sentence_transformers import SentenceTransformer, util
import torch

# Load pre-trained ModernBERT model and tokenizer
model_name = 'answerdotai/ModernBERT-base'  # Changed to ModernBERT
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

# Or use a sentence transformer model directly for better performance
# model = SentenceTransformer('answerdotai/ModernBERT-base') # Example: ModernBERT-base


def get_bert_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)  # Average pooling of token embeddings
    return embeddings


def calculate_similarity(text1, text2):
    embedding1 = get_bert_embedding(text1)
    embedding2 = get_bert_embedding(text2)

    # Calculate cosine similarity
    similarity_score = util.cos_sim(embedding1, embedding2).item()
    return similarity_score


# Example usage
text1 = "This is a bad movie."
text2 = "This is positive sentence."
text3 = "This is a negative sentence."

similarity_1_2 = calculate_similarity(text1, text2)
similarity_1_3 = calculate_similarity(text1, text3)

print(f"Similarity between '{text1}' and '{text2}': {similarity_1_2}")
print(f"Similarity between '{text1}' and '{text3}': {similarity_1_3}")


You are using a model of type modernbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of BertModel were not initialized from the model checkpoint at answerdotai/ModernBERT-base and are newly initialized: ['embeddings.LayerNorm.bias', 'embeddings.LayerNorm.weight', 'embeddings.position_embeddings.weight', 'embeddings.token_type_embeddings.weight', 'embeddings.word_embeddings.weight', 'encoder.layer.0.attention.output.LayerNorm.bias', 'encoder.layer.0.attention.output.LayerNorm.weight', 'encoder.layer.0.attention.output.dense.bias', 'encoder.layer.0.attention.output.dense.weight', 'encoder.layer.0.attention.self.key.bias', 'encoder.layer.0.attention.self.key.weight', 'encoder.layer.0.attention.self.query.bias', 'encoder.layer.0.attention.self.query.weight', 'encoder.layer.0.attention.self.value.bias', 'encoder.layer.0.attention.self.value.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.

Similarity between 'This is a bad movie.' and 'This is positive sentence.': 0.9630316495895386
Similarity between 'This is a bad movie.' and 'This is a negative sentence.': 0.9782860279083252
